In [1]:
import snscrape.modules.twitter as sntwitter #only works with python 3.11 or lower
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import spacy
import emoji

In [2]:
# Define your search topic
query = "#ElectricVehicles since:2024-11-01 until:2024-11-04 lang:en"

tweets = []
limit = 10 # number of tweets to collect, limit per month is 100 posts


for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
    if i > limit:
        break
    tweets.append([tweet.date, tweet.user.username, tweet.content])


Error retrieving https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22%23ElectricVehicles%20since%3A2024-11-01%20until%3A2024-11-04%20lang%3Aen%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rweb_tweet_is_tra

ScraperException: 4 requests to https://twitter.com/i/api/graphql/7jT5GT59P8IFjgxwqnEdQw/SearchTimeline?variables=%7B%22rawQuery%22%3A%22%23ElectricVehicles%20since%3A2024-11-01%20until%3A2024-11-04%20lang%3Aen%22%2C%22count%22%3A20%2C%22product%22%3A%22Latest%22%2C%22withDownvotePerspective%22%3Afalse%2C%22withReactionsMetadata%22%3Afalse%2C%22withReactionsPerspective%22%3Afalse%7D&features=%7B%22rweb_lists_timeline_redesign_enabled%22%3Afalse%2C%22blue_business_profile_image_shape_enabled%22%3Afalse%2C%22responsive_web_graphql_exclude_directive_enabled%22%3Atrue%2C%22verified_phone_label_enabled%22%3Afalse%2C%22creator_subscriptions_tweet_preview_api_enabled%22%3Afalse%2C%22responsive_web_graphql_timeline_navigation_enabled%22%3Atrue%2C%22responsive_web_graphql_skip_user_profile_image_extensions_enabled%22%3Afalse%2C%22tweetypie_unmention_optimization_enabled%22%3Atrue%2C%22vibe_api_enabled%22%3Atrue%2C%22responsive_web_edit_tweet_api_enabled%22%3Atrue%2C%22graphql_is_translatable_rw

In [3]:
# Create DataFrame
df = pd.DataFrame(tweets, columns=["Date", "User", "Tweet"])
df.to_csv("tweets_ai.csv", index=False)
df.head()

,Date,User,Tweet


## 2. Data Cleaning and Preparation

Clean and prepare the tweet text data for sentiment analysis. 
The steps include:

1. Removing URLs, mentions, hashtags (or removing the ‘#’ but keeping the word), emojis and special characters  
2. Converting text to lowercase  
3. Tokenizing the text  
4. Removing stop-words  
5. Lemmatizing the tokens  
6. Optionally rebuilding the cleaned text field  




In [4]:
# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

# Define stopwords
stop_words = set(stopwords.words('english'))

# A function to remove URLs, mentions, hashtags (#), emojis, special chars
def clean_tweet_text(text):
    # Remove URLs
    text = re.sub(r"http\S+|www.\S+", "", text)
    # Remove mentions (@username)
    text = re.sub(r"@\w+", "", text)
    # Remove hashtag symbol (keep the word) – e.g. "#ElectricVehicles" → "ElectricVehicles"
    text = re.sub(r"#", "", text)
    # Demojize (optional) or simply remove emojis
    text = emoji.replace_emoji(text, replace="")
    # Remove punctuation and special characters
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    # Convert to lowercase
    text = text.lower()
    return text

# A function to tokenize, remove stopwords, and lemmatize
def tokenize_lemmatize(text):
    # Tokenize
    tokens = word_tokenize(text)
    # Remove stopwords and tokens of length <= 1
    tokens = [tok for tok in tokens if tok not in stop_words and len(tok) > 1]
    # Lemmatize using spaCy
    doc = nlp(" ".join(tokens))
    lemmas = [token.lemma_ for token in doc]
    return lemmas

# Example usage: load the data
df = pd.read_csv("tweets_evs.csv")   # make sure your filename matches
# Apply cleaning
df['clean_text'] = df['Tweet'].apply(clean_tweet_text)
df['tokens'] = df['clean_text'].apply(tokenize_lemmatize)
# Optionally join tokens back to a cleaned string
df['cleaned_joined'] = df['tokens'].apply(lambda x: " ".join(x))

# Show the first few cleaned rows
df.head()


OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

### Explanation of cleaning steps:
- We start by removing URLs and mentions because they don’t contribute to sentiment in most cases and may bias the text.  
- We remove the “#” symbol but keep the keyword after it so that hashtags still contribute their meaning (for example, ‘ElectricVehicles’ becomes a regular word).  
- Emojis can carry sentiment but for simplicity we remove them here (you could instead convert emojis to text meaning using `emoji.demojize`).  
- Punctuation and special characters are removed to reduce noise.  
- Lowercasing ensures that words like “EVs”, “evs”, and “Evs” are treated the same.  
- Tokenization breaks the text into individual tokens/words.  
- Stop-words removal strips out very common words (“the”, “is”, “and”) that don’t carry sentiment by themselves.  
- Lemmatization reduces words to their base form (“running” → “run”, “better” → “good”), which helps with grouping similar concepts.  
- Finally, we rebuild a cleaned version of the text for downstream analysis (sentiment scoring, word-cloud, etc).  

```python
# Save cleaned data (optional)
df.to_csv("tweets_evs_cleaned.csv", index=False)


## 3. Sentiment Analysis

In this section, we will analyze the sentiment of our cleaned tweets using three different approaches:
1. **TextBlob** – simple polarity-based sentiment scoring  
2. **VADER (Valence Aware Dictionary for sEntiment Reasoning)** – optimized for social media text  
3. *(Optional)* **Transformer model (HuggingFace)** – a pretrained deep learning model for more accurate sentiment classification  

For each tweet, we will compute its sentiment polarity and then visualize the distribution of sentiments (positive, neutral, negative).


In [ ]:

from textblob import TextBlob
import matplotlib.pyplot as plt
import seaborn as sns

# Load cleaned data
df = pd.read_csv("tweets_evs_cleaned.csv")

# Compute polarity: range is [-1.0, 1.0]
df['polarity'] = df['cleaned_joined'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)

# Define sentiment categories
def get_sentiment_label(p):
    if p > 0.05:
        return 'Positive'
    elif p < -0.05:
        return 'Negative'
    else:
        return 'Neutral'

df['sentiment'] = df['polarity'].apply(get_sentiment_label)

# Display sample results
df[['cleaned_joined', 'polarity', 'sentiment']].head()


### Visualizing sentiment distribution

In [ ]:
# Sentiment distribution
sns.set(style="whitegrid")
plt.figure(figsize=(7,5))
sns.countplot(x='sentiment', data=df, order=['Positive', 'Neutral', 'Negative'])
plt.title("Sentiment Distribution (TextBlob)")
plt.xlabel("Sentiment")
plt.ylabel("Number of Tweets")
plt.show()

# Average polarity
avg_sentiment = df['polarity'].mean()
print(f"Average Sentiment Polarity: {avg_sentiment:.3f}")


#### Using VADER (Better for social media text)

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

# Apply VADER to each tweet
df['vader_scores'] = df['cleaned_joined'].apply(lambda x: analyzer.polarity_scores(str(x))['compound'])

# Convert to categorical sentiment
df['vader_sentiment'] = df['vader_scores'].apply(lambda s: 'Positive' if s > 0.05 else ('Negative' if s < -0.05 else 'Neutral'))

# Visualization
plt.figure(figsize=(7,5))
sns.countplot(x='vader_sentiment', data=df, order=['Positive', 'Neutral', 'Negative'], palette='pastel')
plt.title("Sentiment Distribution (VADER)")
plt.xlabel("Sentiment")
plt.ylabel("Number of Tweets")
plt.show()

# Average sentiment score
avg_vader = df['vader_scores'].mean()
print(f"Average VADER Sentiment Score: {avg_vader:.3f}")


### Interpretation of Results

From the TextBlob and VADER analyses, we can observe that:
- The majority of tweets around **#ElectricVehicles** are positive, showing enthusiasm about EV technology and sustainability.
- Neutral tweets mostly share factual updates (e.g., government policy or product announcements).
- Negative tweets often discuss high costs, range anxiety, or charging challenges.

The VADER model produced slightly more neutral classifications compared to TextBlob, which tends to exaggerate positive sentiment. The optional Transformer model provides more context-aware scoring and aligns more closely with nuanced opinions.


## 4. Word Cloud Visualization

To complement the sentiment analysis, we will visualize the most frequent words used in the tweets through a **word cloud**.  
A word cloud shows words sized proportionally to their frequency — larger words appear more often in the dataset.

This helps identify common themes or discussion points around our topic (**#ElectricVehicles**).


In [ ]:
from wordcloud import WordCloud, STOPWORDS

# Combine all cleaned text
text_all = " ".join(df['cleaned_joined'].astype(str))

# Add default and custom stopwords to filter out uninformative words
stopwords = set(STOPWORDS)
custom_stopwords = {"https", "co", "rt", "amp"}  # common Twitter noise
stopwords.update(custom_stopwords)

# Generate the word cloud
wordcloud = WordCloud(width=1000,
                      height=600,
                      background_color='white',
                      stopwords=stopwords,
                      collocations=False,
                      max_words=150).generate(text_all)

# Display it
plt.figure(figsize=(12, 7))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title("Most Frequent Words in #ElectricVehicles Tweets", fontsize=14)
plt.show()


### Interpretation of Themes
- Words such as **“tesla”, “battery”, “charging”, “price”, “sustainability”** may appear most frequently, reflecting public focus on technology, cost, and environmental impact.  
- If policy-related terms appear (e.g., “subsidy”, “government”, “incentive”), they indicate discussion around regulation and infrastructure.  
- The presence of **positive** words (e.g., “love”, “future”, “innovation”) or **negative** ones (e.g., “expensive”, “problem”, “delay”) complements the sentiment analysis findings.

### Summary of Step 4
The word cloud provides a quick visual summary of the conversation landscape and helps identify keywords driving positive or negative sentiment.  
Together with Step 3’s sentiment plots, it gives a comprehensive overview of public mood toward **#ElectricVehicles**.
